In [1]:
from typing import List, Optional

In [5]:
# This class represents the Wedding event
class Wedding:
    def __init__(self, bride_name: str, groom_name: str) -> None:
        # Set the names of the couple getting married
        self.bride_name = bride_name
        self.groom_name = groom_name

        # Create empty lists to hold guests and invitations
        self.confirmed_guest_list: List['Guest'] = []
        self.invitation_list: List['Invitation'] = []

    def send_invitation(self, name: str, email: str, is_special: bool = False) -> None:
        # Send an invitation to a guest (special or regular)

        # If a guest with the same email is already invited, do nothing
        if self.get_guest_by_email(email):
            return

        # Create either a SpecialGuest or a regular Guest
        if is_special:
            guest = SpecialGuest(name, email, self)
        else:
            guest = Guest(name, email, self)

        # Create an invitation and add it to the list
        invitation = Invitation(guest)
        self.invitation_list.append(invitation)

    def retrieve_invitation(self, email: str) -> Optional['Invitation']:
        # Find and return an invitation by the guest's email
        for invitation in self.invitation_list:
            if invitation.guest.email == email:
                return invitation
        return None

    def get_guest_by_email(self, email: str) -> Optional['Guest']:
        # Find and return a guest by email
        for invitation in self.invitation_list:
            if invitation.guest.email == email:
                return invitation.guest
        return None

# This class represents an Invitation
class Invitation:
    def __init__(self, guest: 'Guest') -> None:
        self.guest = guest
        self.status = "pending"  # Can be "pending", "accepted", or "declined"

    def accept(self) -> None:
        self.status = "accepted"

    def decline(self) -> None:
        self.status = "declined"

# This class represents a Guest invited to the wedding
class Guest:
    def __init__(self, name: str, email: str, wedding: Wedding, inviting_guest_email: Optional[str] = None) -> None:
        self.name = name
        self.email = email
        self.wedding = wedding
        self.inviting_guest_email = inviting_guest_email  # Used if invited as a plus-one

    def accept_invitation(self) -> None:
        # Guest accepts the invitation
        invitation = self.wedding.retrieve_invitation(self.email)
        if invitation:
            invitation.accept()
            if self not in self.wedding.confirmed_guest_list:
                self.wedding.confirmed_guest_list.append(self)

    def decline_invitation(self) -> None:
        # Guest declines the invitation
        invitation = self.wedding.retrieve_invitation(self.email)
        if invitation:
            invitation.decline()
            if self in self.wedding.confirmed_guest_list:
                self.wedding.confirmed_guest_list.remove(self)

# This class is a type of Guest who can bring a plus-one
class SpecialGuest(Guest):
    def __init__(self, name: str, email: str, wedding: Wedding) -> None:
        super().__init__(name, email, wedding)
        self.plus_one: Optional[Guest] = None

    def invite_plus_one(self, name: str, email: str) -> None:
        # Invite someone as a plus-one
        if self.plus_one:
            return  # Already has a plus-one

        if not self.wedding.get_guest_by_email(email):
            self.plus_one = Guest(name, email, self.wedding, self.email)
            self.wedding.send_invitation(name, email)

    def uninvite_plus_one(self) -> None:
        # Remove the plus-one and their invitation
        if self.plus_one:
            invitation = self.wedding.retrieve_invitation(self.plus_one.email)
            if invitation in self.wedding.invitation_list:
                self.wedding.invitation_list.remove(invitation)

            if self.plus_one in self.wedding.confirmed_guest_list:
                self.wedding.confirmed_guest_list.remove(self.plus_one)

            self.plus_one = None